# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn import tree

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['figure.figsize'] = 12,4

# 2. Import dataset
* 02_cat_num is imported 
* There are 15 independent variables

In [3]:
df = pd.read_csv('../01 raw data/02_cat_num.csv')
print(df.shape)

(199927, 17)


# 3. categorical
* there are 9 categorical variables 
* each category is replaced with the average Fraud

In [4]:
def cat(df, x, y, z):
    dtree = tree.DecisionTreeClassifier(random_state=42, max_depth=3, min_samples_leaf=2000)
    dtree = dtree.fit(df[['b_'+x]],df[y])
    df[z] = dtree.apply(df[['b_'+x]])
    gb = df.groupby(z).agg({y:'mean'}).reset_index()
    gb['key'] = 1

    gtree = pd.read_csv('../03 rolledup data/'+x+'_b.csv')
    gtree['key'] = 1
    gtree = pd.merge(gtree,gb,how='inner',on='key')
    gtree[x] = gtree[x].fillna('MISSING')

    maxtree = gtree[gtree['b_'+x]>=gtree[y]]
    maxtree = maxtree.groupby(x).agg({y:'max'}).reset_index()
    maxtree.columns = [x,'max']

    mintree = gtree[gtree['b_'+x]<=gtree[y]]
    mintree = mintree.groupby(x).agg({y:'min'}).reset_index()
    mintree.columns = [x,'min']

    gb = pd.merge(maxtree,mintree,how='outer',on=x)
    gb['max'] = gb['max'].fillna(0)
    gb['min'] = gb['min'].fillna(1)
    gb['b_'+x] = (gb['max']+gb['min'])/2
    gb = gb.drop(['min','max'], axis=1)
    gb.to_csv('../03 rolledup data/'+x+'_b.csv', index=False)

In [5]:
for i in ['accountCountry','isUserRegistered','isProxyIP','transactionCurrencyCode','cardType','cvvVerifyResult',
          'paymentInstrumentType','ipState','ipCountryCode']:
    cat(df, i, 'Fraud', 'e_'+i)

print(df.shape)

(199927, 26)


# 4. Import dataset
* 01_clean is imported 
* There are 15 independent variables

In [6]:
df = pd.read_csv('../01 raw data/01_clean.csv')
df['paymentInstrumentAgeInAccount'] = pd.to_numeric(df['paymentInstrumentAgeInAccount'], errors='coerce')
print(df.shape)

(199927, 17)


# 5. numerical
* there are 7 categorical variables
* decision tree is used to bin the variable
* max depth = 3 and min sample size = 1% of observations (2000)

In [7]:
def num(df, x, y, z):
    tempn = df[df[x].notnull()]
    dtree = tree.DecisionTreeClassifier(random_state=42, max_depth=3, min_samples_leaf=2000)
    dtree = dtree.fit(tempn[[x]],tempn[y])
    tempn[z] = dtree.apply(tempn[[x]])
    
    tempm = df[df[x].isnull()]
    tempm[z] = 'MISSING'
    temp = pd.concat([tempm, tempn])
    
    gb = temp.groupby(z).agg({x:['min','max'],y:'mean'}).reset_index()
    gb = gb.drop(z, axis=1)
    gb.columns = [x+'_min', x+'_max', 'b_'+x]
    gb.to_csv('../03 rolledup data/c_'+x+'_b.csv', index=False)

In [8]:
n = pd.DataFrame()
for i in ['localWeekday','localHour','accountAge','transactionAmountUSD','numPaymentRejects1dPerUser',
          'paymentInstrumentAgeInAccount']: 
    num(df, i, 'Fraud', 'b_'+i)

print(df.shape)

(199927, 17)
